In [ ]:
from pyesasky import ESASkyWidget
from pyesasky import Catalogue
from pyesasky import CatalogueDescriptor
from pyesasky import MetadataDescriptor
from pyesasky import MetadataType
from pyesasky import CooFrame

In [ ]:
# instantiating pyESASky instance
esasky = ESASkyWidget()

In [ ]:
# loading pyESASky instance
esasky

In [ ]:
# Go to the Cosmos field in ESASky (as resolved by SIMBAD):
esasky.goToTargetName('Cosmos Field')

In [ ]:
#####################################################
# EX.1 creating a user defined catalogue on the fly #
#####################################################
catalogue = Catalogue('test catalogue name', CooFrame.FRAME_J2000, '#ee2345', 10)

In [ ]:
# adding sources to the catalogue
catalogue.addSource('source name A', '150.44963', '2.24640', 1, [{"name":"Flux 1", "value":"10.5", "type":"STRING" },{"name":"Flux 2", "value":"1.7", "type":"STRING" }])
catalogue.addSource('source name B', '150.54963', '2.34640', 2, [{"name":"Flux 1", "value":"11.5", "type":"STRING" },{"name":"Flux 2", "value":"2.7", "type":"STRING" }])
catalogue.addSource('source name c', '150.34963', '2.44640', 3, [{"name":"Flux 1", "value":"12.5", "type":"STRING" },{"name":"Flux 2", "value":"0.7", "type":"STRING" }])

In [ ]:
# overlay catalogue in pyESASky
esasky.overlayCatalogueWithDetails(catalogue)

In [ ]:
############################################
# EX.2 importing a catalogue from CSV file #
############################################
# CatalogueDescriptor('<catName>', '<HTMLcolor>', <lineWidth>, '<idColumn>', '<nameColumn>', '<RAColumn>', '<DecColumn>', Metadata)
# where:
# - <catName> : name of the catalogue that will be used in pyESASky as label
# - <HTMLcolor> : HTML color. It could be a "Color name", "Hex color code" or "RGB color code"
# - <lineWidth> : width used to draw sources. From 1 to 10
# - <idColumn> : name of the column containing a unique identifier for sources if any. None if not applicable
# - <nameColumn> : name of the column with the name of the source
# - <RAColumn> : name of the RA column in degrees
# - <DecColumn> : name of the Dec column in degrees
#  - Metadata : list of pyesasky.pyesasky.MetadataDescriptor in case it has been defined. [] otherwise.
catalogueDesc =CatalogueDescriptor('my test', 'yellow', 5, 'id', 'name', 'ra', 'dec', [])

In [ ]:
# parse, import and overlay a catalogue from a CSV
esasky.overlayCatalogueFromCSV('./testcat', ',', catalogueDesc, 'J2000')

In [ ]:
###################################################################
# EX.3 importing a catalogue from AstropyTable using Gaia archive #
###################################################################
from astroquery.gaia import Gaia

In [ ]:
job = Gaia.launch_job("select top 10\
 ra, dec, source_id, designation, ref_epoch,ra_dec_corr,astrometric_n_obs_al,matched_observations,duplicated_source,phot_variable_flag \
 from gaiadr2.gaia_source order by source_id", verbose=True)


In [ ]:
myGaiaData = job.get_results()
print(myGaiaData)

In [ ]:
job.get_data()

In [ ]:
# overlayCatalogueFromAstropyTable('<catName>', '<cooFrame>', <HTMLcolor>, '<(astropy.table)>', '<RAColumn>', '<DecColumn>', '<nameColumn>')
# where:
# - <catName> : name of the catalogue that will be used in pyESASky as label
# - <HTMLcolor> : HTML color. It could be a "Color name", "Hex color code" or "RGB color code"
# - <lineWidth> : width used to draw sources. From 1 to 10
# - <idColumn> : name of the column containing a unique identifier for sources if any. None if not applicable
# - <nameColumn> : name of the column with the name of the source
# - <RAColumn> : name of the RA column in degrees
# - <DecColumn> : name of the Dec column in degrees

esasky.overlayCatalogueFromAstropyTable('Gaia DR2', 'J2000', '#a343ff', 5, myGaiaData, '','','')

In [ ]:
# Import the VizieR Astroquery module
from astroquery.vizier import Vizier

In [ ]:
# Search for 'The XMM-Newton survey of the COSMOS field (Brusa+, 2010)':
catalog_list = Vizier.find_catalogs('Brusa+, 2010')
print({k:v.description for k,v in catalog_list.items()})

In [ ]:
# Get the above list of catalogues:
Vizier.ROW_LIMIT = -1
catalogs = Vizier.get_catalogs(catalog_list.keys())
print(catalogs)

In [ ]:
# Access one table: 
Brusa = catalogs['J/ApJ/716/348/table2']
print(Brusa)

In [ ]:
# Visualise the table in ESASky:
esasky.overlayCatalogueFromAstropyTable('Brusa', CooFrame.FRAME_J2000, '#00ff00', 5, Brusa, 'RAJ2000','DEJ2000','Name')

In [ ]:
# Go to the LMC in ESASky (as resolved by SIMBAD):
esasky.goToTargetName('LMC')

In [ ]:
# Search for 'The HIZOA-S survey':
catalog_list2 = Vizier.find_catalogs('HIZOA-S survey 2016') #HIZOA-S survey 2016
print({k:v.description for k,v in catalog_list2.items()})

In [ ]:
# Get the above list of catalogues:
Vizier.ROW_LIMIT = -1
# Vizier.COLUMN_LIMIT = 20 Can't find the way to get all the columns rather than just the default columns. Going to try the TAP+ module
catalog = Vizier.get_catalogs(catalog_list2.keys())
print(catalog)

In [ ]:
# Access the catalogue table: 
HIZOA = catalog['J/AJ/151/52/table2'] #
print(HIZOA)

In [ ]:
# Visualise the table in ESASky:
###### NOTE: NOT PLOTTING GALACTIC COORDS CORRECTLY
esasky.overlayCatalogueFromAstropyTable('HIZOA', CooFrame.FRAME_GALACTIC, '#0000ff', 7, HIZOA, 'GLON','GLAT','HIZOA')

In [ ]:
# TRYING THE SAME BUT USING THE TAP/TAP+ ASTROQUERY MODULE:
# Import the TAP/TAP+ Astroquery module
from astroquery.utils.tap.core import TapPlus

In [ ]:
vizier = TapPlus(url="http://tapvizier.u-strasbg.fr/TAPVizieR/tap")

In [ ]:
tables = vizier.load_tables(only_names=True)
for table in (tables):
  print(table.get_qualified_name())

In [ ]:
#ONLY TAP+ compatible, so doesn't seem to work
table = vizier.load_table('viz7."J/AJ/128/16/table2"')
for column in (table.get_columns()):
    print(column.get_name())

In [ ]:
# This works in TOPCAT to download the whole table: SELECT * FROM "J/AJ/128/16/table2"
# This also works in TOPCAT : SELECT * FROM viz7."J/AJ/128/16/table2"

job = vizier.launch_job("SELECT * FROM "'viz7."J/AJ/128/16/table2"'"")
#This also works:
#job = vizier.launch_job("SELECT * FROM "+str('viz7."J/AJ/128/16/table2"')+"")
print(job)

In [ ]:
Koribalski = job.get_results()
print(Koribalski['HIPASS', 'RAJ2000', 'DEJ2000'])

In [ ]:
# Visualise the table in ESASky:
esasky.overlayCatalogueFromAstropyTable('Koribalski', CooFrame.FRAME_J2000, '#ff0000', 6, Koribalski, 'RAJ2000','DEJ2000','HIPASS')